In [1]:
import polars as pl
import altair as alt

In [2]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [4]:
len(df)

699208

In [5]:
def vyvoj_vyskytu(hledani, popisek):
    
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("vyskytu")
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("vyskytu").sum()
    ).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("vyskytu") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [6]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)
    

In [7]:
graf(vyvoj_vyskytu("(?i)v televizi","v televizi"))

alt.Chart(...)

In [8]:
graf(vyvoj_vyskytu("(?i)v r[aá]diu","v rádiu"))

alt.Chart(...)

In [9]:
graf(vyvoj_vyskytu("(?i)v rozhlase","v rozhlase"))

alt.Chart(...)

In [10]:
graf(vyvoj_vyskytu("(?i)v novinách","v novinách"))

alt.Chart(...)

In [11]:
graf(vyvoj_vyskytu("(?i)v knížce","v knížce"))

alt.Chart(...)

In [12]:
graf(vyvoj_vyskytu("(?i)ve{0,1} \w{0,20}\s{0,1}románu","v románu"))

alt.Chart(...)

In [13]:
def hledej(text, n=3, delka=200, frejm=df):
    nalezy = frejm.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [14]:
import re

In [15]:
hledej("v románu")

Kdo: Poslanec Miloslav Ransdorf
Kde: 1998ps_stenprot_013schuz_s013456.htm
Kdy: 1999-06-08
Co: tní péče je příkladem, který příliš vhodný není. Někdy se mi zdá, že přístup evropských byrokratů je podobný tomu, jak postupoval výbor na oslavu 70letého mírového panování císaře Františka Josefa I. v románu Muž bez vlastností Roberta Musila. Vypukla první světová válka, císař zemřel, ale výbor pro oslavu jeho 70letého mírového panování pracoval dál. I když se změnily některé podmínky, přesto určité norm
***
Kdo: Poslanec Vladimír Zlínský
Kde: 2021ps_stenprot_074schuz_s074087.htm
Kdy: 2023-09-06
Co: r informací prostřednictvím digitalizace společnosti, analýza těchto dat, systémy umělé inteligence, efektivní centrální řízení společnosti v míře, která byla dříve těžko představitelná a známá spíše v románu George Orwella 1984. Záměrně tento román tady zmiňuji, já jsem ho nedávno četl - přiznám se, že jsem velký čtenář, ale měl jsem deficit, že jsem ho nečetl, protože jsem byl před tím varován, 

In [16]:
graf(vyvoj_vyskytu("(?i)v knize","v knize"))

alt.Chart(...)

In [17]:
graf(vyvoj_vyskytu("(?i)v časopis","v časopisu"))

alt.Chart(...)

In [18]:
graf(vyvoj_vyskytu("(?i)","v časopisu"))

alt.Chart(...)

In [19]:
graf(vyvoj_vyskytu("(?i)na internetu","na internetu"))

alt.Chart(...)

In [20]:
graf(vyvoj_vyskytu("(?i)whatsapp","whatsapp"))

alt.Chart(...)

In [21]:
import re

In [22]:
def jak_napsal(retezec):
    return re.findall(r"(?i)jak (napsala{0,1}|píše) \w{1,20} \w{1,20} \w{1,20}",retezec)

In [23]:
jak_napsal("protože jak napsala František Novák v knize")

['napsala']

In [24]:
def v_knize(retezec):
    return re.findall(r"jak (se píše|stojí) v knize \w{1,20} \w{1,20} \w{1,20}",retezec)

In [25]:
len(df) / 1000

699.208

In [26]:
%%timeit
df.sample(729).with_columns(pl.col('text').map_elements(jak_napsal, return_dtype=pl.List(pl.String)).alias('jak_napsal'))

64.5 ms ± 4.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
df = df.with_columns(pl.col('text').map_elements(jak_napsal, return_dtype=pl.List(pl.String)).alias('jak_napsal'))

In [28]:
df.explode('jak_napsal').group_by('jak_napsal').len().sort(by='len',descending=True)

jak_napsal,len
str,u32
null,699162
"""píše""",33
"""napsal""",13
"""napsala""",2


In [29]:
df = df.with_columns(pl.col('text').map_elements(v_knize, return_dtype=pl.List(pl.String)).alias('v_knize'))

In [30]:
def spisovatel(retezec):
    return [' '.join(x.split(' ')[1:]) for x in re.findall(r"(?i)spisovatel\w{0,5} \w{1,15} \w{1,15}\.{0,1} \w{1,15}",retezec)]

In [31]:
spisovatel('Slovy spisovatele Marka P. Twaina: srát na slávu jak na placatý kámen.')

['Marka P. Twaina']

In [ ]:
df = df.with_columns(pl.col('text').map_elements(spisovatel, return_dtype=pl.List(pl.String)).alias('spisovatelstvo'))

In [ ]:
df.group_by('spisovatelstvo').len().sort(by='len',descending=True)

In [ ]:
graf(vyvoj_vyskytu("Kar\w{1,5} Čap\w{1,10}","Karel Čapek"))

In [ ]:
graf(vyvoj_vyskytu("Milan\w{0,5} Kunder\w{1,10}","Milan Kundera"))

In [ ]:
graf(vyvoj_vyskytu("Franz\w{0,5} Kafk\w{1,10}","Franz Kafka"))

In [ ]:
graf(vyvoj_vyskytu("Jaros\w{1,10} Haš[ek]{1,10}","Jaroslav Hašek"))

In [ ]:
graf(vyvoj_vyskytu("Božen\w{1,10} Němcov{1,10}","Božena Němcová"))

In [ ]:
graf(vyvoj_vyskytu("Jaros\w{1,10} Seifert{1,10}","Jaroslav Seifert"))

In [ ]:
graf(vyvoj_vyskytu("Juli\w{1,10} Fučí","Julius Fučík"))

In [ ]:
graf(vyvoj_vyskytu("Václav\w{0,10} Hav[el]\w{1,10}","Václav Havel"))

In [ ]:
graf(vyvoj_vyskytu("Jan\w{0,10} Nerud",""))

In [ ]:
graf(vyvoj_vyskytu("Kar\w{1,10} Hyn\w{1,10} Mách\w{1,10}",""))

In [ ]:
graf(vyvoj_vyskytu("Goethe",""))

In [ ]:
graf(vyvoj_vyskytu("Bibl\w{1,10}",""))

In [ ]:
graf(vyvoj_vyskytu("[oO]rwell",""))

In [ ]:
graf(vyvoj_vyskytu("Kar\w{1,10} Havl\w{1,10} Borovsk\w{1,10}",""))

In [ ]:
graf(vyvoj_vyskytu("Aldou\w{1,10} Huxl\w{1,10}",""))

In [ ]:
graf(vyvoj_vyskytu("Masaryk\w{0,10}",""))

In [ ]:
graf(vyvoj_vyskytu("Arendt\w{0,10}",""))

In [ ]:
graf(vyvoj_vyskytu("(?i)superman\w{0,10}",""))

In [ ]:
graf(vyvoj_vyskytu("(?i)batman\w{0,10}",""))

In [ ]:
graf(vyvoj_vyskytu("(?i)disney\w{0,10}",""))

In [ ]:
graf(vyvoj_vyskytu("(?i)Keynes\w{0,10}",""))

In [ ]:
graf(vyvoj_vyskytu("(?i)orwell\w{0,10}",""))

In [ ]:
df.filter(pl.col("text").str.contains("(?i)orwell")).group_by("mluvci").len().sort(by="len",descending=True)

In [ ]:
hledej("(?i)orwell")

In [ ]:
graf(vyvoj_vyskytu("(?i)Friedman\w{0,10}",""))

In [ ]:
graf_ekonomu = alt.Chart(
    pl.concat(
        [
            vyvoj_vyskytu("Keynes","Keynes"),
            vyvoj_vyskytu("Friedman","Friedman"),
            vyvoj_vyskytu("Marx","Marx")
        ]
    ).to_pandas(),
    width=500,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N", sort=['Marx','Keynes','Friedman'])
)

graf_ekonomu

## Syntéza médií

In [ ]:
media_pokupe = pl.concat(
        [
            vyvoj_vyskytu(r"(?i)[^\w]v televiz","v televizi"),
            vyvoj_vyskytu(r"(?i)[^\w](v rozhlas|v r[áa]diu)","v rozhlase / rádiu"),
            vyvoj_vyskytu(r"(?i)[^\w]v časopis","v časopise"),
            vyvoj_vyskytu(r"(?i)[^\w]v novinách","v novinách"),
            vyvoj_vyskytu(r"(?i)[^\w]ve{0,1} \w{0,20}\s{0,1}knize","v knize"),
            vyvoj_vyskytu(r"(?i)[^\w]ve{0,1} \w{0,20}\s{0,1}románu","v románu"),
            vyvoj_vyskytu(r"(?i)[^\w]na internetu","na internetu"),
            vyvoj_vyskytu(r"(?i)[^\w]na facebooku","na Facebooku"),
            vyvoj_vyskytu(r"(?i)[^\w]youtu","YouTube"),
            vyvoj_vyskytu(r"(?i)tweet","Twitter"),
            vyvoj_vyskytu(r"DVTV","DVTV"),
        ]
)

In [ ]:
graf(media_pokupe)

In [ ]:
media_pokupe

In [ ]:
media_pokupe_fin = media_pokupe.join(
    media_pokupe.group_by(
        "datum"
    ).agg(
        pl.col("pomer").max()
    ).with_columns(
        pl.lit(True).alias("max")
    ), 
    how="left",
    on=["datum","pomer"]
).with_columns(
    pl.when(pl.col('max').is_null()).then(pl.lit(False)).otherwise(pl.lit(True))
)



In [ ]:
print(media_pokupe.select(pl.col('co')).unique().to_series().to_list())

In [ ]:
poradi_medii = ['v novinách','v knize','v časopise','v televizi','v rozhlase / rádiu','na internetu']

In [ ]:
graf_medii = alt.Chart(
    media_pokupe_fin.filter(pl.col('co').is_in(poradi_medii)).to_pandas(),
    width=330,
    height=60,
    title=alt.Title(
        "Zmínky o médiích v poslaneckých projevech",
        subtitle=["Červeně jsou vyznačeny roky, v nichž šlo o médium dominantní."]
    )
).mark_bar(
    width=2
).encode(
    alt.X('datum:T', title=None),
    alt.Y('pomer:Q', axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', tickCount=3, labelExpr="datum.label * 100 + ' %'", orient='right', title=None)),
    alt.Row('co:N', sort=poradi_medii, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap'), title=None),
    alt.Color('max:N', legend=None, scale=alt.Scale(range=['#6B8EAA','#D55F50']))
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

graf_medii

In [ ]:
me_to_neurazi(
    graf=graf_medii,
    kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', 
    soubor='ruzna_media',
    slozka_na_serveru='grafy-stenoprotokoly'
)

## Konkrétní noviny

In [ ]:
graf(pl.concat(
        [
            vyvoj_vyskytu(r"(?i) v noviná","v novinách"),
            vyvoj_vyskytu(r"Rudé\w{0,5} práv","Rudé právo"),
            vyvoj_vyskytu(r"MFD","MFD"),
            vyvoj_vyskytu(r"Národní politi[kc]","Národní politika"),
            vyvoj_vyskytu(r" v Přítomnosti","v Přítomnosti"),
            vyvoj_vyskytu(r"České\w{0,3} slov[ouě]","České slovo"),
            vyvoj_vyskytu(r"Svobodn\w{0,3} slov[ouě]","Svobodné slovo"),
            vyvoj_vyskytu(r"Pražsk\w{0,5} novin","Pražské noviny"),
            vyvoj_vyskytu(r"Lidov\w{0,5} novin","Lidové noviny"),
            vyvoj_vyskytu(r"Hospod\w{0,8} novin","Hospodářské noviny"),
            vyvoj_vyskytu(r"(?i)mlad\w{0,8} front\w{0,8} dnes","Mladá fronta DNES"),
            vyvoj_vyskytu(r"[\w,] Respekt","Respekt"),
            vyvoj_vyskytu(r"[\w,] Reflex","Reflex"),
            vyvoj_vyskytu(r"[\w,] Reflex","Reflex"),
            vyvoj_vyskytu(r" v Blesku","v Blesku"),
        ]
))